# Import libraries

In [1]:
from llama_index.core import SimpleDirectoryReader
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import PyPDF2
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

True

# Initialize the directory reader and embedding model

In [3]:
# Initialize the directory reader
input_dir_path = "../profile/"
loader = SimpleDirectoryReader(input_dir=input_dir_path, required_exts=[".pdf"], recursive=True)
docs = loader.load_data()

# Initialize the embedding model
embed_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/Users/suryakiransureshkumar/Documents/ProfilePal/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Connect to pinecone

In [4]:
pinecone_api_key = os.getenv("PINECONE_API_KEY")
index_name = os.getenv("INDEX_NAME")

In [5]:
pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index(index_name)

In [6]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

# Generate embeddings and upsert them into Pinecone

In [7]:
vectors =[]
for doc in docs:
    # Generate embedding for the document text
    embedding = embed_model.encode(doc.text)
    
    # Upsert the embedding into Pinecone
    vectors.append(
        {
            "id": doc.id_, 
            "values": embedding, 
            "metadata": {"text":doc.text}
        })
    # index.upsert([(doc.id_, embedding,{"text":doc.text})])

index.upsert(vectors=vectors)
print("Embeddings have been upserted into the Pinecone index.")

Embeddings have been upserted into the Pinecone index.
